# Lab 04: Quantum Noise and Decoherence

In this lab, you will explore the effects of noise and decoherence on quantum circuits. Unlike classical computers, quantum computers are extremely sensitive to environmental interactions, which can lead to errors. Understanding these noise effects is crucial for developing error mitigation strategies and quantum error correction techniques.

## Learning Objectives
After completing this lab, you should be able to:
1. Understand different types of quantum noise models
2. Implement and simulate noisy quantum circuits
3. Visualize the effects of noise on the Bloch sphere
4. Measure the impact of noise on quantum algorithm performance
5. Apply basic error mitigation techniques

## Prerequisites
- Basic understanding of quantum circuits and gates
- Familiarity with the Bloch sphere representation of qubits
- Experience with quantum measurement and probabilities

## Exercise 1: Setup and Environment Preparation

First, let's import the necessary libraries and set up our quantum environment.

In [ ]:
# Import necessary libraries
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram, plot_bloch_multivector, plot_bloch_vector
from qiskit.quantum_info import Statevector
from qiskit_aer.noise import NoiseModel, pauli_error, depolarizing_error
from qiskit.quantum_info import state_fidelity

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Enable inline plotting
%matplotlib inline

## Exercise 2: Simulating a Perfect vs. Noisy Quantum Circuit

Let's start by comparing the behavior of a simple quantum circuit with and without noise. This will help us understand the basic impact of noise on measurement results.

**Task**: Create a simple quantum circuit that prepares a qubit in the |0⟩ state and then measures it. Run the circuit on both a perfect simulator and a noisy simulator, and compare the results.

In [ ]:
# YOUR CODE HERE: Create a quantum circuit with one qubit and one classical bit
qc = QuantumCircuit(1, 1)

# YOUR CODE HERE: Add a measurement operation
qc.measure_all()

# YOUR CODE HERE: Draw the circuit
display(qc.draw('mpl'))

# Run on a perfect simulator
# YOUR CODE HERE: Run the circuit on a perfect simulator with 1000 shots
ideal_backend = Aer.get_backend('qasm_simulator')
ideal_job = ideal_backend.run(qc, shots=1000)
ideal_counts = ideal_job.result().get_counts()

# Create a custom noise model
# YOUR CODE HERE: Create a noise model with bit-flip error on measurements
# The error probability should be 5% (0.05)
noise_model = NoiseModel()
error_prob = 0.05
bit_flip_error = pauli_error([('X', error_prob), ('I', 1-error_prob)])
noise_model.add_all_qubit_quantum_error(bit_flip_error, ['measure'])

# Run with the noise model
# YOUR CODE HERE: Run the circuit with the noise model
noisy_backend = Aer.get_backend('qasm_simulator')
noisy_job = noisy_backend.run(qc, noise_model=noise_model, shots=1000)
noisy_counts = noisy_job.result().get_counts()

# Display and compare the results
# YOUR CODE HERE: Plot and analyze results from both simulations
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

plot_histogram(ideal_counts, title="Perfect Simulator", ax=axs[0])
plot_histogram(noisy_counts, title="Noisy Simulator (Bit-Flip Noise)", ax=axs[1])

plt.tight_layout()
plt.show()

### Understanding Bit-Flip Noise

In the noise model you created above, you used a Pauli-X error to model bit-flip noise. Let's understand what this means:

1. **Bit-Flip Noise**: This type of noise randomly flips the state of a qubit from |0⟩ to |1⟩ or vice versa.
   - Mathematically represented by applying a Pauli-X gate with some probability
   - The notation `pauli_error([('X', 0.05), ('I', 0.95)])` means:
     - With 5% probability: Apply Pauli-X gate (bit flip)
     - With 95% probability: Apply Identity operation (no change)

2. **Real-World Causes of Bit-Flip Errors**:
   - Environmental noise affecting qubit state
   - Imperfect control pulses during gate operations
   - Readout errors during measurement

## Exercise 3: Noise Effects on Superposition

Now, let's see how noise affects quantum circuits that involve superposition.

**Task**: Create a circuit that applies a Hadamard gate to put a qubit into superposition, then measures it. Compare the results with and without noise.

In [ ]:
# YOUR CODE HERE: Create a circuit with Hadamard gate and measurement
qc_super = QuantumCircuit(1, 1)
qc_super.h(0)
qc_super.measure_all()

# YOUR CODE HERE: Draw the circuit
display(qc_super.draw('mpl'))

# Create a noise model with both bit-flip and depolarizing noise
# YOUR CODE HERE: Create a noise model with:
# - 5% bit-flip error on measurements
# - 2% depolarizing error on single-qubit gates
super_noise_model = NoiseModel()

# Bit-flip error on measurements
bit_flip_error_super = pauli_error([('X', 0.05), ('I', 0.95)])
super_noise_model.add_all_qubit_quantum_error(bit_flip_error_super, ['measure'])

# Depolarizing error on single-qubit gates
depol_error_super = depolarizing_error(0.02, 1)
super_noise_model.add_all_qubit_quantum_error(depol_error_super, ['u1', 'u2', 'u3'])

# Run on perfect and noisy simulators
# YOUR CODE HERE: Run on both simulators and compare results
ideal_super_job = ideal_backend.run(qc_super, shots=1000)
ideal_super_counts = ideal_super_job.result().get_counts()

noisy_super_job = noisy_backend.run(qc_super, noise_model=super_noise_model, shots=1000)
noisy_super_counts = noisy_super_job.result().get_counts()

# Display and compare the results
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

plot_histogram(ideal_super_counts, title="Perfect Simulator", ax=axs[0])
plot_histogram(noisy_super_counts, title="Noisy Simulator (With Noise)", ax=axs[1])

plt.tight_layout()
plt.show()

# Analyze how far the noisy results deviate from the ideal 50-50 split
# YOUR CODE HERE: Calculate and print probability analysis
ideal_prob = np.array([ideal_super_counts.get('0', 0), ideal_super_counts.get('1', 0)]) / 1000
noisy_prob = np.array([noisy_super_counts.get('0', 0), noisy_super_counts.get('1', 0)]) / 1000

# Chi-squared distance
chi_squared_distance = np.sum((ideal_prob - noisy_prob)**2 / ideal_prob)

print("Ideal probabilities:", ideal_prob)
print("Noisy probabilities:", noisy_prob)
print("Chi-squared distance:", chi_squared_distance)

### Understanding Depolarizing Noise

In your enhanced noise model, you added depolarizing error to single-qubit gates. Let's understand what this means:

1. **Depolarizing Noise**: This type of noise randomly transforms a qubit's state toward the maximally mixed state (the center of the Bloch sphere).
   - With probability p: The qubit is replaced with a completely mixed state
   - With probability (1-p): The qubit remains unchanged

2. **Mathematical Representation**: A depolarizing channel with probability p can be described as:
   - With probability (1-p): The qubit state remains unchanged
   - With probability p/3: The X gate is applied
   - With probability p/3: The Y gate is applied
   - With probability p/3: The Z gate is applied

3. **Visual Effect**: On the Bloch sphere, depolarizing noise causes the state vector to shrink toward the center.

## Exercise 4: Visualizing Depolarizing Noise on the Bloch Sphere

Let's visualize how depolarizing noise affects quantum states on the Bloch sphere.

**Task**: Create a function to apply depolarizing noise to a quantum state and visualize the effect on the Bloch sphere.

In [ ]:
# YOUR CODE HERE: Define a function to apply depolarizing noise
def apply_depolarizing_noise(initial_state, noise_strength):
    """
    Apply depolarizing noise to a quantum state
    
    Args:
        initial_state: Initial quantum state vector [x, y, z]
        noise_strength: Value between 0 and 1 indicating noise strength
        
    Returns:
        Noisy state vector [x', y', z']
    """
    # YOUR CODE HERE: Implement the function
    from qiskit.quantum_info import DensityMatrix

    # Convert initial state to DensityMatrix object
    initial_density = DensityMatrix(initial_state)

    # Apply depolarizing channel
    noisy_density = initial_density.depolarize(noise_strength)

    # Return the resulting state vector
    return noisy_density.to_statevector()

# YOUR CODE HERE: Choose an initial state to visualize
# For example, the |+⟩ state: [1, 0, 0]
initial_state = [1, 0, 0]  # |+⟩ state

# YOUR CODE HERE: Apply noise at different strengths and visualize
noise_strengths = [0, 0.1, 0.3, 0.5]
fig, axs = plt.subplots(1, len(noise_strengths), figsize=(15, 3))

for i, strength in enumerate(noise_strengths):
    noisy_state = apply_depolarizing_noise(initial_state, strength)
    plot_bloch_vector(noisy_state, ax=axs[i])
    axs[i].set_title(f"Noise Strength = {strength}")

plt.show()

## Exercise 5: Phase-Flip Noise

In addition to bit-flip and depolarizing noise, another important type of quantum noise is phase-flip noise, which randomly changes the phase of a qubit.

**Task**: Create a noise model with phase-flip errors and observe its effect on a superposition state.

In [ ]:
# YOUR CODE HERE: Create a noise model with phase-flip errors
phase_flip_model = NoiseModel()
phase_flip_1q = pauli_error([('Z', 0.05), ('I', 0.95)])  # 1-qubit phase-flip error
phase_flip_model.add_all_qubit_quantum_error(phase_flip_1q, ["id", "u1", "u2", "u3"])
# For 2-qubit gates, we need a 2-qubit error
phase_flip_2q = pauli_error([('ZI', 0.025), ('IZ', 0.025), ('II', 0.95)])  # 2-qubit phase-flip error
phase_flip_model.add_all_qubit_quantum_error(phase_flip_2q, ["cx"])

# Create a circuit to test phase-flip noise
# The circuit should create a superposition, then apply the noise,
# then apply another Hadamard to convert phase difference to bit difference
# YOUR CODE HERE: Create and run the circuit
qc_phase = QuantumCircuit(1, 1)
qc_phase.h(0)
qc_phase.measure_all()

# Run with phase-flip noise
noisy_phase_job = noisy_backend.run(qc_phase, noise_model=phase_flip_model, shots=1000)
noisy_phase_counts = noisy_phase_job.result().get_counts()

# Compare with a circuit without noise
# YOUR CODE HERE: Run the same circuit without noise and compare
ideal_phase_job = ideal_backend.run(qc_phase, shots=1000)
ideal_phase_counts = ideal_phase_job.result().get_counts()

# Display the results
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

plot_histogram(ideal_phase_counts, title="Perfect Simulator", ax=axs[0])
plot_histogram(noisy_phase_counts, title="Noisy Simulator (Phase-Flip Noise)", ax=axs[1])

plt.tight_layout()
plt.show()

### Understanding Phase-Flip Noise

1. **Phase-Flip Noise**: This type of noise randomly changes the phase of a qubit.
   - Mathematically represented by applying a Pauli-Z gate with some probability
   - The notation `pauli_error([('Z', 0.05), ('I', 0.95)])` means:
     - With 5% probability: Apply Pauli-Z gate (phase flip)
     - With 95% probability: Apply Identity operation (no change)

2. **Effect on Superposition**: Phase-flip noise is particularly damaging to quantum algorithms because it disrupts the phase relationships that are crucial for quantum interference.

3. **Detection**: Phase-flip errors can't be detected by direct measurement in the computational basis. We need to convert phase information to bit information (e.g., by applying a Hadamard gate) before measurement.

## Exercise 6: Noise Effects on a Bell State

Now let's see how noise affects entangled states, which are even more sensitive to noise than single-qubit states.

**Task**: Create a circuit that prepares a Bell state, and observe how different types of noise affect the entanglement.

In [ ]:
# YOUR CODE HERE: Create a circuit that prepares a Bell state
bell_circuit = QuantumCircuit(2, 2)
bell_circuit.h(0)         # Apply Hadamard to first qubit
bell_circuit.cx(0, 1)     # Apply CNOT gate with control=first qubit, target=second qubit
bell_circuit.measure([0, 1], [0, 1])

# YOUR CODE HERE: Draw the circuit
print("Bell state circuit:")
display(bell_circuit.draw('mpl'))

# Create different noise models
# YOUR CODE HERE: Create noise models with bit-flip, phase-flip, and depolarizing noise

# 1. Bit-flip noise model
bit_flip_model = NoiseModel()
bit_flip_1q = pauli_error([('X', 0.05), ('I', 0.95)])  # 1-qubit bit-flip error
bit_flip_model.add_all_qubit_quantum_error(bit_flip_1q, ["id", "u1", "u2", "u3"])
# For 2-qubit gates, we need a 2-qubit error
bit_flip_2q = pauli_error([('XI', 0.025), ('IX', 0.025), ('II', 0.95)])  # 2-qubit bit-flip error
bit_flip_model.add_all_qubit_quantum_error(bit_flip_2q, ["cx"])

# 2. Phase-flip noise model
phase_flip_model = NoiseModel()
phase_flip_1q = pauli_error([('Z', 0.05), ('I', 0.95)])  # 1-qubit phase-flip error
phase_flip_model.add_all_qubit_quantum_error(phase_flip_1q, ["id", "u1", "u2", "u3"])
# For 2-qubit gates, we need a 2-qubit error
phase_flip_2q = pauli_error([('ZI', 0.025), ('IZ', 0.025), ('II', 0.95)])  # 2-qubit phase-flip error
phase_flip_model.add_all_qubit_quantum_error(phase_flip_2q, ["cx"])

# 3. Depolarizing noise model
depol_model = NoiseModel()
depol_1q = depolarizing_error(0.05, 1)  # 1-qubit depolarizing error
depol_2q = depolarizing_error(0.05, 2)  # 2-qubit depolarizing error
depol_model.add_all_qubit_quantum_error(depol_1q, ["id", "u1", "u2", "u3"])
depol_model.add_all_qubit_quantum_error(depol_2q, ["cx"])

# Run the Bell state circuit with different noise models
# YOUR CODE HERE: Run simulations and compare results
# Run without noise
perfect_backend = Aer.get_backend('qasm_simulator')
perfect_bell_job = perfect_backend.run(bell_circuit, shots=1000)
perfect_bell_counts = perfect_bell_job.result().get_counts()

# Run with bit-flip noise
noisy_backend = Aer.get_backend('qasm_simulator')
bit_flip_bell_job = noisy_backend.run(bell_circuit, noise_model=bit_flip_model, shots=1000)
bit_flip_bell_counts = bit_flip_bell_job.result().get_counts()

# Run with phase-flip noise
phase_flip_bell_job = noisy_backend.run(bell_circuit, noise_model=phase_flip_model, shots=1000)
phase_flip_bell_counts = phase_flip_bell_job.result().get_counts()

# Run with depolarizing noise
depol_bell_job = noisy_backend.run(bell_circuit, noise_model=depol_model, shots=1000)
depol_bell_counts = depol_bell_job.result().get_counts()

# Display the results
fig, axs = plt.subplots(2, 2, figsize=(12, 8))

plot_histogram(perfect_bell_counts, title="Bell State - No Noise", ax=axs[0, 0])
plot_histogram(bit_flip_bell_counts, title="Bell State - Bit-Flip Noise", ax=axs[0, 1])
plot_histogram(phase_flip_bell_counts, title="Bell State - Phase-Flip Noise", ax=axs[1, 0])
plot_histogram(depol_bell_counts, title="Bell State - Depolarizing Noise", ax=axs[1, 1])

plt.tight_layout()
plt.show()

### Understanding Noise Effects on Entanglement

Entangled states like Bell states are particularly sensitive to noise for several reasons:

1. **Non-local Effects**: Noise affecting just one qubit of an entangled pair can affect the joint state of both qubits.

2. **Decoherence**: Noise causes quantum states to lose their coherence, which is essential for maintaining entanglement.

3. **Measurement Statistics**: In a perfect Bell state, the measurement outcomes of the two qubits should be perfectly correlated. Noise disrupts this correlation.

## Exercise 7: Error Mitigation with Repetition

One simple approach to mitigating quantum errors is to run the circuit multiple times and take the average result. This can help reduce the impact of random noise.

**Task**: Implement a simple error mitigation technique by running a noisy circuit multiple times and comparing the average result with the ideal result.

In [ ]:
# YOUR CODE HERE: Create a circuit for testing error mitigation
mitigation_circuit = QuantumCircuit(1, 1)
mitigation_circuit.h(0)
mitigation_circuit.measure_all()

# YOUR CODE HERE: Create a noise model
mitigation_noise_model = NoiseModel()
mitigation_depol = depolarizing_error(0.1, 1)  # 10% depolarizing error
mitigation_noise_model.add_all_qubit_quantum_error(mitigation_depol, ["u1", "u2", "u3"])

# Run the circuit once with noise
# YOUR CODE HERE: Run the circuit once with 1000 shots
single_run_job = noisy_backend.run(mitigation_circuit, noise_model=mitigation_noise_model, shots=1000)
single_run_counts = single_run_job.result().get_counts()

# Run the circuit multiple times with noise and average the results
# YOUR CODE HERE: Run the circuit 10 times with 100 shots each, then average
num_trials = 10
shots_per_trial = 100
all_counts = []

for _ in range(num_trials):
    job = noisy_backend.run(mitigation_circuit, noise_model=mitigation_noise_model, shots=shots_per_trial)
    result = job.result().get_counts()
    all_counts.append(result)

# Average the results
from collections import Counter

# Combine all counts
combined_counts = Counter()
for count in all_counts:
    combined_counts.update(count)

# Average the counts
avg_counts = {key: val / num_trials for key, val in combined_counts.items()}

# Compare the results
# YOUR CODE HERE: Compare single-run results, averaged results, and ideal results
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

plot_histogram(single_run_counts, title="Single Run with Noise", ax=axs[0])
plot_histogram(avg_counts, title="Averaged Results (10 Trials)", ax=axs[1])

# Ideal results for comparison
ideal_mitigation_job = ideal_backend.run(mitigation_circuit, shots=1000)
ideal_mitigation_counts = ideal_mitigation_job.result().get_counts()
plot_histogram(ideal_mitigation_counts, title="Ideal Results", ax=axs[2])

plt.tight_layout()
plt.show()

## Reflection Questions

1. How do bit-flip and phase-flip errors differ in terms of their effect on quantum states?

2. Why is quantum error correction more challenging than classical error correction?

3. How does depolarizing noise affect the Bloch sphere representation of a qubit?

4. Why might entangled states be more sensitive to noise than non-entangled states?

5. Can you think of any applications where quantum noise might actually be useful?

## Summary

In this lab, you have explored:
- Different types of quantum noise models (bit-flip, phase-flip, depolarizing)
- The effects of noise on single-qubit and multi-qubit circuits
- Visualization of noise effects on the Bloch sphere
- Basic error mitigation techniques

Understanding quantum noise and decoherence is crucial for the development of practical quantum computers. As quantum systems scale up, error correction and mitigation strategies become increasingly important for running reliable quantum algorithms.